In [29]:
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import googletrans
import time
import numpy as np
import csv
import os

import requests
import json

import urllib.request
import json



In [2]:
translator = googletrans.Translator()
# temp = translator.translate(e.text, dest='ko')
# htmls = requests.get()
driver = webdriver.Chrome(ChromeDriverManager().install())

# Papago

In [32]:
def papago(text):
    client_id = "3cpmjj5fb5"
    client_secret = "ySoVv7Rqkp8rzLyPl89fgPkxNmGkrUcFXBJjXDWv"
    custom_dict = "111351d0-b3de-4cf8-8d59-5c05b50c988f"
    encText = urllib.parse.quote(text)
    data = "source=en&target=ko&glossaryKey=" + custom_dict  + "&text=" + encText

    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        d = json.loads(result)
        return(d['message']['result']['translatedText'])
        # return (response_body.decode('utf-8'))
        # return (type(response_body.decode('utf-8')))
    else:
        return ("Error Code:" + rescode)
    

In [4]:
driver.get('https://www.webnovel.com/book/reincarnated-with-the-strongest-system_19720038005035905')
res = driver.find_elements_by_xpath('//script[@type="application/ld+json"]')[1]

aaa = res.get_attribute("innerHTML")
aaa = aaa.replace('\n', '')
info_result = json.loads(aaa, strict=False)
# aaa = aaa.replace(' ', '')
# print(aaa)
# aaa = aaa.replace('},\n{', '}\n{')
# bbb = aaa.split("\n")
# print(bbb[1])


# print(aaa)
# print(aaa[:50])
# print(aaa.rstrip(']').lstrip('[')[:50])



info_result[0]['name']
# print(info_result[0])

# jsons = []
# for line in aaa:
#     jsons.append(json.loads(line), strict=False)
    
# jsons

# def getGroups(s):
#     current = ''
#     count = 0
#     for c in s:
#         if c == '{':
#             count += 1
#         elif c == '}':
#             count -=1 
#         current += c
#         if count == 0:
#             yield current
#             current = ''

# [json.loads(js) for js in getGroups(aaa)]


# tempA ='{"name":"hyen", "age":"18"}'
# json.loads(tempA)
# res.GetAttribute("innerHTML");

# aaa.find("\n")

{'@context': 'https://schema.org', '@type': 'Book', 'mainEntityOfPage': 'https://www.webnovel.com/book/reincarnated-with-the-strongest-system_19720038005035905', 'headline': 'Reincarnated With The Strongest System', 'name': 'Reincarnated With The Strongest System', 'genre': 'Fantasy', 'image': {'@type': 'ImageObject', 'url': '//img.webnovel.com/bookcover/19720038005035905/150/150.jpg?coverUpdateTime=1671718712705&amp;imageMogr2/quality/80', 'height': 424, 'width': 300}, 'bookFormat': 'https://schema.org/EBook', 'datePublished': '2021-04-01T02:09:17.000Z', 'dateModified': '2023-01-31T23:14:39.000Z', 'author': {'@type': 'Person', 'name': 'Elyon'}, 'copyrightHolder': 'Elyon / Webnovel', 'publisher': {'@type': 'Organization', 'name': 'Webnovel', 'logo': {'@type': 'ImageObject', 'url': 'https://www.yueimg.com/overseam/img/common/logo.3cebb.png', 'width': 300}}, 'description': '“Darkness cannot drive out darkness, only Light can do that,&#34; the Goddess Amalthea said as she held William in 

In [5]:
# def get_info_box(url):
    

In [6]:
# ccc = "[abc[]defg]"
# print(ccc.rstrip(']').lstrip('['))

# Scraping main

In [37]:
def scrape(url):
    result = {}
    
    res = driver.get(url)
    
    ### 동적추출
    target = driver.find_elements_by_xpath('//script[@type="application/ld+json"]')[1]

    raw_json = target.get_attribute("innerHTML")
    raw_json = raw_json.replace('\n', '')
    info_box_result = json.loads(raw_json, strict=False)
    
    result['startDate'] = info_box_result[0]['datePublished']
    result['original'] = info_box_result[0]['mainEntityOfPage']
    
    
    ### 정적추출
    htmls = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    bsObject = bs(htmls.content, 'html.parser')
    
  
 
    title = bsObject.find('h1').text
    result['title'] = title
    print(title, papago(title))
    
    
    result['author'] =bsObject.select_one('body > div.page > div.det-hd.mb48 > div > div > div._mn.g_col._8.pr > address > h2 > a').text
    # print(author.text)
    
    result['genre'] = bsObject.select_one('body > div.page > div.det-hd.mb48 > div > div > div._mn.g_col._8.pr > h2 > a > span').text
    # print(genre.text)
    
    result['synopsis'] = driver.find_element_by_xpath('//*[@id="about"]/div[1]/div[1]/p').text
    # print(synopsis.text)
    
    #태그     
    tags = bsObject.select('#about > div.j_tagWrap > div > div.m-tags > p')
    tags_result = []
    for tag in tags:
        tags_result.append(tag.text.replace("#", "").strip())
    result['tags'] = tags_result
    
    #파일 저장
    if os.path.isfile('data.json'):
        with open('data.json', 'r')as f:
            info = json.load(f)
        info['data'].append(result)
        with open('data.json', 'w')as f:
            json.dump(info, f)
    else :
        with open('data.json', 'w')as f:
            root = {"data": [result]}
            json.dump(root, f)
    return;

In [38]:
scraped_result = scrape('https://www.webnovel.com/book/reincarnated-with-the-strongest-system_19720038005035905')

Reincarnated With The Strongest System RWTSS 최강의 시스템 RWTS로 환생


In [9]:
json.dumps(scraped_result, indent=4)

'null'

In [10]:
# with open('data.json', 'w')as f:
#     json.dump(scraped_result,f,indent=4)

In [11]:
# aaa = driver.find_elements_by_xpath('//script[@type="application/ld+json"]')
# for item in aaa:
#     print(item)

In [12]:
# with open('data.json', "r") as json_file:
    # json_data = json.load(json_file)
    # print(json_data)

In [13]:
# aaa = driver.find_elements_by_xpath('//script[@type="application/ld+json"]')
# for item in drvier:
#     print(item)

In [14]:
tempa = {"data" : [{"a": 1, "b": 2},{"a": 3, "b": 4}]}

In [15]:
tempa["data"].append({"a": 5, "b": 6})
json.dumps(tempa["data"])

'[{"a": 1, "b": 2}, {"a": 3, "b": 4}, {"a": 5, "b": 6}]'